In [1]:
#import lxml.html
#import requests
import numpy as np
import pandas as pd
pd.set_option('display.max_columns', None)
import time
import math
import re
import sqlite3
import json
import plotly.graph_objects as go
import plotly.express as px
#from bs4 import BeautifulSoup as bs4
#from selenium.webdriver import Chrome
#from selenium import webdriver
#from selenium.webdriver.common.by import By
from datetime import datetime, timedelta

from sqlalchemy import MetaData, text, Column, Integer, String, ForeignKey, Table, create_engine, Float, Boolean, DateTime
from sqlalchemy.orm import relationship, backref, sessionmaker
from sqlalchemy.ext.declarative import declarative_base

import optimize_lineup as ol
import fantasy_utils as fu

In [2]:
conn = sqlite3.connect('fantasy_data.db')
#conn.create_function('sqrt', 1, math.sqrt)
import logging
logging.basicConfig(level=logging.DEBUG)

def safe_sqrt(x):
    try:
        return math.sqrt(x)
    except Exception as e:
        logging.error(f"Error computing sqrt for {x}: {e}")
        return None  # or some other appropriate value or handling

conn.create_function('sqrt', 1, safe_sqrt)

In [3]:
team_list = ['Lima Time', 'Ugly Spuds', 'Harvey', 'Roiders', 'Charmer', 'Wu Tang', '9 Grand Kids', 'Brewbirds', 'Trouble', 'Lil Trump', 'Dirty Birds', 'Young Guns'] 
eligibility_url_dict = {"9 Grand Kids":'30', 'Brewbirds':'3', 'Charmer':'36', 'Dirty Birds':'41', "Harveys Wallbangers":'4', 'Lil Trump & the Ivanabees':'27', 'Lima Time!':'38', 
                        'Roid Ragers':'44', 'Trouble with the Curve':'1', 'Ugly Spuds':'29', 'Wiscompton Wu-Tang':'42', 'Young Guns':'45', "Mom's Cookin":'47'}
stat_list = ['BA', 'HR', 'R', 'RBI', 'SB', 'ERA', 'Sv+Hld', 'SO', 'W', 'WHIP']

In [4]:
opening_week = int(datetime.strftime(pd.Timestamp('2024-03-28'), '%W')) - 1
cur_week = int(datetime.strftime(pd.to_datetime('2024-05-06'), '%W'))
period = cur_week - opening_week
current_period = period + 1
recent_year = pd.read_sql("SELECT max(year) year FROM roster", conn).iloc[0]['year']
recent_week = pd.read_sql(f"SELECT max(week) week FROM roster where year={recent_year}", conn).iloc[0]['week']
opening_week, cur_week, period, current_period, recent_week

(12, 19, 7, 8, 18)

In [5]:
pd.read_sql("SELECT name FROM sqlite_master", conn)

,name
0,hitting
1,ix_hitting_index
2,hitters
3,sqlite_autoindex_hitters_1
4,players2022
5,owners
6,drafted
7,roster
8,stats
9,players2023


In [6]:
fp = fu.Fantasy_Projections()
ids = fp.load_id_map()

In [7]:
def load_fangraphs_ros_projections(y):
    fg_ros_proj_h = pd.read_csv(f'data/{y}-fangraphs-leaderboard-projections-ros-h.csv')
    fg_ros_proj_h.rename(columns={'SO':'K'}, inplace=True)
    fg_ros_proj_p = pd.read_csv(f'data/{y}-fangraphs-leaderboard-projections-ros-p.csv')
    fg_ros_proj_p.rename(columns={'H':'Ha', 'BB':'BBa'},inplace=True)
    fg_ros_proj_p = fg_ros_proj_p[fg_ros_proj_p['Name']!='Shohei Ohtani']
    fg_ros_proj_p['SvHld'] = fg_ros_proj_p['SV']+fg_ros_proj_p['HLD']
    fg_ros_proj = pd.concat([fg_ros_proj_h, fg_ros_proj_p])
    return fg_ros_proj

In [8]:
fg_ros_proj = load_fangraphs_ros_projections(2024)
fg_ros_proj.columns

Index(['Name', 'Team', 'G', 'PA', 'AB', 'H', '1B', '2B', '3B', 'HR', 'R',
       'RBI', 'BB', 'IBB', 'K', 'HBP', 'SF', 'SH', 'GDP', 'SB', 'CS', 'AVG',
       'BB%', 'K%', 'BB/K', 'OBP', 'SLG', 'wOBA', 'OPS', 'ISO', 'Spd', 'BABIP',
       'UBR', 'wSB', 'wRC', 'wRAA', 'wRC+', 'BsR', 'Fld', 'Off', 'Def', 'WAR',
       'ADP', 'InterSD', 'InterSK', 'IntraSD', 'PlayerId', 'W', 'L', 'QS',
       'ERA', 'GS', 'SV', 'HLD', 'IP', 'TBF', 'Ha', 'ER', 'BBa', 'SO', 'K/9',
       'BB/9', 'K/BB', 'HR/9', 'K-BB%', 'WHIP', 'LOB%', 'GB%', 'HR/FB', 'FIP',
       'RA9-WAR', 'SvHld'],
      dtype='object')

In [65]:
fg_ros_proj = load_fangraphs_ros_projections(2024)
fg_ros_proj['BA'] = fg_ros_proj['AVG']
proj = pd.read_sql(f"SELECT p.CBSNAME, p.IDFANGRAPHS PlayerId, o.owner, \
        e.all_pos, e.posC, e.pos1B, e.pos2B, e.pos3B, e.posSS, \
        e.posMI, e.posCI, e.posOF, e.posDH, e.posSP, e.posRP, e.posP, r.* \
        FROM roster r \
        LEFT JOIN players p On (p.cbsid=r.cbsid) \
        LEFT JOIN owners o On (r.owner_id=o.owner_id) \
        LEFT JOIN (SELECT * FROM eligibility WHERE year={recent_year} AND week={recent_week}) e On (r.cbsid=e.cbsid) \
        WHERE r.year={recent_year} AND r.week={recent_week}", conn).merge(fg_ros_proj, on='PlayerId', how='inner').fillna(0)
proj.loc[(proj['PA']>0), 'type'] = 'h'
proj.loc[(proj['PA']==0), 'type'] = 'p'


h_mask = (proj['AB']>0) #(proj['Pos'].isin(['C', '1B', '2B', '3B', 'LF', 'CF', 'RF', 'SS', 'DH']))
hitters = (proj['type']=='h') #(proj['Pos'].isin(['C', '1B', '2B', '3B', 'LF', 'CF', 'RF', 'SS', 'DH']))
p_mask = (proj['IP']>0) #(proj['Pos'].isin(['SP', 'RP']))
pitchers = (proj['type']=='p') #(proj['Pos'].isin(['SP', 'RP']))

lgBA = proj[h_mask]['H'].sum()/proj[h_mask]['AB'].sum()
proj.loc[hitters, 'zlgBA'] = (proj[hitters]['H'] - (proj[hitters]['AB'] * lgBA))

lgERA = proj[p_mask]['ER'].sum()/proj[p_mask]['IP'].sum()*9
proj.loc[pitchers, 'zlgERA'] = (((proj[pitchers]['ER']*9) - (proj[pitchers]['IP']*lgERA))*-1)

lgWHIP = (proj[p_mask]['BBa'].sum()+proj[p_mask]['Ha'].sum())/proj[p_mask]['IP'].sum()
proj.loc[pitchers, 'zlgWHIP'] = (((proj.loc[pitchers]['Ha']+proj.loc[pitchers]['BBa'])-(proj.loc[pitchers]['IP']*lgWHIP))*-1)

q = proj[h_mask][['HR', 'RBI', 'R', 'SB', 'H', 'AB']].describe().T[['mean', 'std']].T.to_dict()
q['BA'] = {'mean':proj[h_mask]['H'].sum()/proj[h_mask]['AB'].sum(), 'std':proj[h_mask]['BA'].std()}
q.update(proj[p_mask][['W', 'SO', 'SvHld', 'IP', 'ER', 'BBa', 'Ha']].describe().T[['mean', 'std']].T.to_dict())
q['ERA'] = {'mean':(proj[p_mask]['ER'].sum()/proj[p_mask]['IP'].sum())*9, 'std':(proj[p_mask]['ER']/proj[p_mask]['IP']*9).std()}
q['WHIP'] = {'mean':(proj[p_mask]['BBa'].sum()+proj[p_mask]['Ha'].sum())/proj[p_mask]['IP'].sum(), 'std':((proj[p_mask]['BBa']+proj[p_mask]['Ha'])/proj[p_mask]['IP']).std()}
q['zlgBA'] = {'mean':proj[h_mask]['zlgBA'].mean(), 'std':proj[h_mask]['zlgBA'].std()}
q['zlgERA'] = {'mean':proj[p_mask]['zlgERA'].mean(), 'std':proj[p_mask]['zlgERA'].std()}
q['zlgWHIP'] = {'mean':proj[p_mask]['zlgWHIP'].mean(), 'std':proj[p_mask]['zlgWHIP'].std()}

for stat in ['R', 'HR', 'RBI', 'SB', 'zlgBA']:
    proj[stat] = proj[stat].astype(float)
    proj.loc[hitters, 'z'+stat] = proj[hitters].apply(lambda row: fp.big_board(row, stat, q), axis=1)

proj.loc[hitters & (proj['zlgBA'].isna()), 'zlgBA'] = proj['zlgBA'].min()-.01
proj.loc[hitters & (proj['zzlgBA'].isna()), 'zzlgBA'] = proj['zzlgBA'].min()-.01
proj.loc[hitters, 'BIGAAh'] = proj[hitters]['zR'] + proj[hitters]['zRBI'] + proj[hitters]['zHR'] + proj[hitters]['zSB'] + proj[hitters]['zzlgBA']

for stat in ['W', 'SO', 'SvHld', 'zlgERA', 'zlgWHIP']:
    proj.loc[pitchers, 'z'+stat] = proj[pitchers].apply(lambda row: fp.big_board(row, stat, q), axis=1)

proj.loc[pitchers & (proj['zzlgERA'].isna()), 'zzlgERA'] = proj['zzlgERA'].min()-.01
proj.loc[pitchers & (proj['zzlgWHIP'].isna()), 'zzlgWHIP'] = proj['zzlgWHIP'].min()-.01
proj.loc[pitchers, 'BIGAAp'] = proj[pitchers]['zW']+proj[pitchers]['zSO']+proj[pitchers]['zSvHld']+proj[pitchers]['zzlgERA']+proj[pitchers]['zzlgWHIP']

proj.loc[hitters, 'BIGAAp'] = 0
proj.loc[pitchers, 'BIGAAh'] = 0
proj['z'] = proj['BIGAAh']+proj['BIGAAp']

proj = proj.reset_index(drop=True)

In [66]:
proj.groupby('owner').agg({'cbsid':'count', 'z':'sum'}).sort_values('z', ascending=False)

,cbsid,z
owner,,
Lima Time!,33,26.685643
9 Grand Kids,33,26.159060
Brewbirds,33,20.859221
Harveys Wallbangers,32,12.322869
Charmer,32,12.144822
Ugly Spuds,32,6.897237
Trouble with the Curve,33,5.788284
Roid Ragers,33,-6.219946
Dirty Birds,33,-12.911853


In [327]:
proj['proj_type'] = 'ros'
proj['week'] = 18
proj.loc[proj['IP']==0, ['IP', 'W', 'SO', 'SvHld', 'BBa', 'Ha', 'ERA', 'WHIP', 'ER']] = [None]*9
proj.loc[proj['AB']==0, ['AB', 'R', 'H', 'HR', 'RBI', 'BB', 'SB', 'BA']] = [None]*8

In [300]:
params = proj[['cbsid', 'year', 'week', 'proj_type', 'AB', 'R', 'H', 'HR', 'RBI', 'BB', 'SB', 'BA', \
               'IP', 'W', 'SO', 'SvHld', 'BBa', 'Ha', 'ERA', 'WHIP', 'ER', 'z']]\
                .sort_values('z', ascending=False).to_dict(orient='records')

stmt = "INSERT INTO projections (cbsid, year, week, proj_type, AB, R, H, HR, RBI, BB, SB, BA, IP, W, SO, [Sv+Hld], BBa, Ha, ERA, WHIP, ER, z) VALUES (:cbsid, :year, :week, :proj_type, :AB, :R, :H, :HR, :RBI, :BB, :SB, :BA, :IP, :W, :SO, :SvHld, :BBa, :Ha, :ERA, :WHIP, :ER, :z)"

cursor = conn.cursor()

for param in params:
    cursor.execute(stmt, param)

conn.commit()

In [9]:
pd.read_sql(f"SELECT r.*, o.owner FROM roster r INNER JOIN owners o On (r.owner_id=o.owner_id) \
        WHERE year=2024 AND week=16", conn)\
    .merge(proj[['CBSNAME', 'cbsid']], on='cbsid', how='left', indicator=True).query('_merge=="left_only"')\
    .merge(ids[['CBSID', 'IDFANGRAPHS', 'CBSNAME']], left_on='cbsid', right_on='CBSID', how='left')#.to_dict(orient='records')

NameError: name 'proj' is not defined

In [18]:
params = [
    {'IDFANGRAPHS': '31431', 'cbsid':29075715, 'Name': 'Jordan Beck'},
    {'IDFANGRAPHS': '31349', 'cbsid':29084556, 'Name': 'Victor Scott II'},
    {'IDFANGRAPHS': '30161', 'cbsid':29077337, 'Name': 'Carlos Rodriguez'},
    {'IDFANGRAPHS': '29576', 'cbsid':29086560, 'Name': 'Ben Rice'},
    {'IDFANGRAPHS': '31595', 'cbsid':28920011, 'Name': 'Brooks Lee'},
]

stmt = "INSERT INTO players (cbsid, CBSNAME, IDFANGRAPHS) VALUES (:cbsid, :Name, :IDFANGRAPHS)"


cursor = conn.cursor()

for param in params:
    cursor.execute(stmt, param)

conn.commit()

In [10]:
players = pd.read_sql(f"SELECT r.*, p.CBSNAME, p.IDFANGRAPHS, p.FANGRAPHSNAME FROM players p LEFT JOIN roster r On (p.cbsid=r.cbsid) \
        WHERE year={recent_year} AND week={recent_week}", conn)
players_without_fangraphsid = players[(players['IDFANGRAPHS'].isna()) | (players['IDFANGRAPHS'].str.startswith('sa'))]
players_without_fangraphsid

,cbsid,owner_id,year,week,pos,decision,CBSNAME,IDFANGRAPHS,FANGRAPHSNAME
91,29092535,36,2024,17,SP,start,Ryan Walker,None,None
127,26615406,41,2024,17,SP,Sit,Tink Hence,sa3014709,Tink Hence
325,29075709,42,2024,17,SP,Sit,Jacob Misiorowski,sa3020644,Jacob Misiorowski


In [12]:
params = fg_ros_proj[fg_ros_proj['Name'].isin(players_without_fangraphsid['CBSNAME'].tolist())][['Name', 'Team', 'PlayerId']]\
    .merge(players_without_fangraphsid[['CBSNAME', 'cbsid']], left_on='Name', right_on='CBSNAME', how='left')\
    [['CBSNAME', 'cbsid', 'PlayerId', 'Team']].to_dict(orient='records')
params

[{'CBSNAME': 'Tink Hence',
  'cbsid': 26615406,
  'PlayerId': 'sa3014709',
  'Team': 'STL'},
 {'CBSNAME': 'Jacob Misiorowski',
  'cbsid': 29075709,
  'PlayerId': 'sa3020644',
  'Team': 'MIL'}]

In [164]:
cursor = conn.cursor()

params = [
    {'cbsid':3151554, 'PlayerId':'20778'},
    {'cbsid':2836680, 'PlayerId':'23401'}
]

stmt = "UPDATE players SET IDFANGRAPHS = :PlayerId WHERE cbsid = :cbsid"

for param in params:
    cursor.execute(stmt, param)

conn.commit()

In [14]:
pd.read_sql("SELECT cbsid, CBSNAME, IDFANGRAPHS, FANGRAPHSNAME FROM players WHERE cbsid=28920005", conn)

,cbsid,CBSNAME,IDFANGRAPHS,FANGRAPHSNAME
0,28920005.0,Jackson Holliday,31781,Jackson Holliday


In [12]:
pd.read_sql("SELECT * FROM stats WHERE cbsid=2901324 and year=2024 ORDER BY week", conn)#.drop_duplicates()

,cbsid,year,week,R,RBI,HR,SB,H,AB,W,SO,SvHld,ER,Ha,BBa,IP,outs
0,2901324,2024,1,1.0,2.0,0.0,1.0,3.0,10.0,None,None,None,None,None,None,None,None
1,2901324,2024,2,3.0,0.0,0.0,0.0,4.0,16.0,None,None,None,None,None,None,None,None
2,2901324,2024,3,6.0,5.0,2.0,0.0,9.0,24.0,None,None,None,None,None,None,None,None
3,2901324,2024,4,3.0,3.0,2.0,1.0,9.0,24.0,None,None,None,None,None,None,None,None
4,2901324,2024,5,6.0,3.0,1.0,3.0,10.0,21.0,None,None,None,None,None,None,None,None
5,2901324,2024,6,5.0,5.0,2.0,0.0,6.0,27.0,None,None,None,None,None,None,None,None
6,2901324,2024,7,6.0,7.0,3.0,2.0,11.0,21.0,None,None,None,None,None,None,None,None
7,2901324,2024,8,3.0,2.0,1.0,2.0,4.0,16.0,None,None,None,None,None,None,None,None
8,2901324,2024,9,3.0,6.0,2.0,2.0,10.0,28.0,None,None,None,None,None,None,None,None
9,2901324,2024,10,4.0,2.0,0.0,2.0,5.0,24.0,None,None,None,None,None,None,None,None


In [10]:
# Get all of Ohtani's stats
ohtani_stats = pd.read_sql("SELECT * FROM stats WHERE cbsid=2901324 and year=2024 AND AB IS NOT NULL ORDER BY week", conn).drop_duplicates()
# Set the pitching stats to NULL
ohtani_stats.loc[:, ['W', 'SO','SvHld', 'ER', 'Ha', 'BBa', 'IP', 'outs']] = None
params = ohtani_stats.to_dict(orient='records')
# Delete his stats from the table
cursor = conn.cursor()
stmt = "DELETE FROM stats WHERE year=2024 AND cbsid=2901324"
cursor.execute(stmt)
conn.commit()
# Insert the stats back into the table
stmt = "INSERT INTO stats (cbsid, year, week, R, RBI, HR, SB, H, AB, W, SO, SvHld, ER, Ha, BBa, IP, outs) VALUES (:cbsid, :year, :week, :R, :RBI, :HR, :SB, :H, :AB, :W, :SO, :SvHld, :ER, :Ha, :BBa, :IP, :outs)"
for param in params:
    cursor.execute(stmt, param)
conn.commit()

In [24]:

params = ohtani_stats.to_dict(orient='records')
stmt = "INSERT INTO stats (cbsid, year, week, R, RBI, HR, SB, H, AB, W, SO, SvHld, ER, Ha, BBa, IP, outs) VALUES (:cbsid, :year, :week, :R, :RBI, :HR, :SB, :H, :AB, :W, :SO, :SvHld, :ER, :Ha, :BBa, :IP, :outs)"
cursor.execute(stmt, param)
conn.commit()

In [46]:
cursor = conn.cursor()
#for num in range(2,17):
    
params = [
    {'cbsid':2901324, 'year':2024, 'pos':'DH', 'decision':'start', 'owner_id':36, 'week':num},
]

stmt = "INSERT INTO stats (cbsid, year, week, R, RBI, HR, SB, H, AB, W, SO, SvHld, ER, Ha, BBa, IP, outs) VALUES (:cbsid, :year, :week, :R, :RBI, :HR, :SB, :H, :AB, :W, :SO, :SvHld, :ER, :Ha, :BBa, :IP, :outs)"

for param in params:
    #cursor.execute(stmt, param)

conn.commit()

### Get ROS Projections

In [8]:
url = 'https://www.fangraphs.com/projections?pos=all&stats=bat&type=steamerr&statgroup=fantasy&fantasypreset=roto5x5'
data = requests.get(url)

DEBUG:urllib3.connectionpool:Starting new HTTPS connection (1): www.fangraphs.com:443
DEBUG:urllib3.connectionpool:https://www.fangraphs.com:443 "GET /projections?pos=all&stats=bat&type=steamerr&statgroup=fantasy&fantasypreset=roto5x5 HTTP/1.1" 200 None


In [71]:
fg_ros_proj = load_fangraphs_ros_projections(2024)

In [74]:
proj = pd.read_sql(f"SELECT p.CBSNAME, p.IDFANGRAPHS PlayerId, o.owner, e.all_pos, e.posC, e.pos1B, e.pos2B, e.pos3B, e.posSS, \
        e.posMI, e.posCI, e.posOF, e.posDH, e.posSP, e.posRP, e.posP, r.* \
        FROM roster r \
        LEFT JOIN players p On (p.cbsid=r.cbsid) \
        LEFT JOIN owners o On (r.owner_id=o.owner_id) \
        LEFT JOIN (SELECT * FROM eligibility WHERE year={recent_year} AND week={recent_week}) e On (r.cbsid=e.cbsid) \
        WHERE r.year={recent_year} AND r.week={recent_week}", conn).merge(fg_ros_proj, on='PlayerId', how='inner').fillna(0)
proj.loc[(proj['PA'].notna()) & (proj['all_pos'].isna()), 'all_pos'] = 'DH'
proj.loc[(proj['PA'].isna()) & (proj['all_pos'].isna()), 'all_pos'] = 'P'
proj.shape

(392, 92)

In [151]:
pd.read_sql("SELECT * FROM players WHERE CBSNAME='Junior Caminero'", conn)

,cbsid,CBSNAME,IDPLAYER,PLAYERNAME,BIRTHDATE,FIRSTNAME,LASTNAME,TEAM,LG,POS,IDFANGRAPHS,FANGRAPHSNAME,MLBID,MLBNAME,RETROID,BREFID,NFBCID,NFBCNAME,ESPNID,ESPNNAME,BPID,YAHOOID,YAHOONAME,MSTRBLLNAME,BATS,THROWS,FANTPROSNAME,LASTCOMMAFIRST,ROTOWIREID,FANTRAXID,FANTRAXNAME,ROTOWIRENAME,ALLPOS,NFBCLASTFIRST,ACTIVE
0,28839811.0,Junior Caminero,caminju01,Junior Caminero,7/25/2003,Junior,Caminero,TB,AL,3B,28163,Junior Caminero,691406.0,Junior Caminero,None,caminju01,11875.0,Junior Caminero,None,Junior Caminero,147956.0,None,Junior Caminero,None,R,R,Junior Caminero,"Caminero, Junior",17704.0,*05xch*,Junior Caminero,Junior Caminero,3B,"Caminero, Junior",Y


In [228]:
proj.loc[proj['PA']>0, 'posDH'] = 5
proj.loc[proj['PA'].isna(), 'posDH'] = 0
proj.loc[proj['PA'].isna(), 'posP'] = 5
proj.loc[proj['PA']>0, 'posP'] = 0

In [229]:
#e = pd.read_sql('SELECT * FROM eligibility WHERE year=2024 AND week=17', conn)
proj[[p for p in proj.columns if p.startswith('pos') and len(p)>3]] = proj[[p for p in proj.columns if p.startswith('pos') and len(p)>3]].fillna(0)
proj.all_pos = ''
for pos in [p for p in proj.columns if p.startswith('pos') and len(p)>3]:
    proj.loc[proj[pos]>=5, 'all_pos'] += pos.replace('pos','')+','

proj['all_pos'] = proj['all_pos'].apply(lambda x: x[:-1])
proj.loc[proj['PA']>0, 'posDH'] = 5
proj.loc[proj['PA'].isna(), 'posDH'] = 0
proj

,CBSNAME,PlayerId,owner,all_pos,posC,pos1B,pos2B,pos3B,posSS,posMI,posCI,posOF,posDH,posSP,posRP,posP,cbsid,owner_id,year,week,pos,decision,Name,Team,G,PA,AB,H,1B,2B,3B,HR,R,RBI,BB,IBB,SO,HBP,SF,SH,GDP,SB,CS,AVG,BB%,K%,BB/K,OBP,SLG,wOBA,OPS,ISO,Spd,BABIP,UBR,wSB,wRC,wRAA,wRC+,BsR,Fld,Off,Def,WAR,ADP,InterSD,InterSK,IntraSD,W,L,QS,ERA,GS,SV,HLD,IP,TBF,ER,K/9,BB/9,K/BB,HR/9,K-BB%,WHIP,LOB%,GB%,HR/FB,FIP,RA9-WAR,SvHld
0,Yainer Diaz,23003,Lima Time!,"C,1B,CI,DH",5.0,5.0,0.0,0.0,0.0,0.0,5.0,0.0,5.0,0.0,0.0,0.0,2829012,38,2024,16,C,start,Yainer Diaz,HOU,44.7100,187.531,176.435,47.8917,30.9685,9.0536,0.7145,7.155100,22.2160,26.5315,8.0638,0.0000,31.9171,1.5002,1.2808,0.2511,NaN,0.4467,0.1449,0.271441,0.043000,0.170197,0.252648,0.306791,0.452516,0.327451,0.759307,0.181075,2.98391,0.293823,-0.323,-0.248521,24.5334,2.62467,113.1310,-0.571521,-0.161,2.298080,0.731865,0.835788,111.120003,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,Josh Naylor,18839,Lima Time!,"1B,CI,DH",0.0,5.0,0.0,0.0,0.0,0.0,5.0,0.0,5.0,0.0,0.0,0.0,2184448,38,2024,16,1B,start,Josh Naylor,CLE,62.3000,268.418,241.598,65.1243,39.4299,13.3512,0.1493,12.193900,34.3353,40.2932,22.1982,3.7579,43.6327,2.6842,1.7567,0.1807,NaN,3.8015,1.2739,0.269556,0.082700,0.162555,0.508751,0.335549,0.477469,0.346793,0.813018,0.207913,3.37555,0.282253,-0.470,-0.171959,39.2691,7.91057,127.6460,-0.641959,0.057,8.005989,-4.757382,1.234140,127.699997,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,Ketel Marte,13613,Lima Time!,"2B,MI,DH",0.0,0.0,5.0,0.0,1.0,5.0,0.0,0.0,5.0,0.0,0.0,0.0,1963334,38,2024,16,2B,start,Ketel Marte,ARI,62.9600,285.670,252.840,70.2262,43.1150,15.3615,2.5291,9.220700,40.2845,33.9772,27.4814,1.9997,48.8488,2.5710,1.8824,0.8943,NaN,3.8724,1.0543,0.277749,0.096200,0.170998,0.562581,0.352132,0.467916,0.353527,0.820048,0.190167,5.22144,0.310218,-0.054,-0.142126,43.3322,9.95817,129.4560,-0.196126,-0.421,9.662642,-0.619320,1.941820,114.059998,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,Alex Bregman,17678,Lima Time!,"3B,CI,DH",0.0,0.0,0.0,5.0,0.0,0.0,5.0,0.0,5.0,0.0,0.0,0.0,2184351,38,2024,16,3B,start,Alex Bregman,HOU,63.8900,284.169,246.688,64.1153,40.1112,12.8455,1.3484,9.810200,38.2650,35.3416,31.5711,0.2842,36.5591,3.4100,1.9104,0.5895,NaN,1.2993,0.4103,0.259905,0.111100,0.128653,0.863563,0.349449,0.442212,0.346239,0.791661,0.182307,3.70413,0.268533,-0.423,-0.421467,41.4475,8.24886,126.0300,-0.844467,-0.509,7.775545,0.448960,1.801520,95.160004,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,David Hamilton,27531,Lima Time!,"2B,SS,MI,DH",0.0,0.0,10.0,0.0,45.0,55.0,0.0,0.0,5.0,0.0,0.0,0.0,3158363,38,2024,16,SS,start,David Hamilton,BOS,46.2000,196.785,175.266,40.3192,26.3979,8.5272,1.2697,4.124400,23.9255,19.0437,18.1632,0.0000,49.1622,1.3775,1.3855,0.5924,NaN,15.2019,3.6590,0.230045,0.092300,0.249827,0.369455,0.305108,0.363784,0.296396,0.668892,0.133738,7.05043,0.293396,0.049,1.244560,20.8547,-2.13511,84.8724,1.293560,-0.757,-2.175606,0.572904,0.492673,750.130005,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
388,Dane Dunning,19409,Young Guns,"SP,P",0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,12.0,2.0,5.0,2250614,45,2024,16,SP,sit,Dane Dunning,TEX,16.0514,NaN,NaN,20.9074,NaN,NaN,NaN,2.531980,10.1585,NaN,7.2449,NaN,21.5032,1.1509,NaN,NaN,NaN,NaN,NaN,0.245133,0.077332,0.229524,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.299982,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.187386,370.0

### Retrospective Trade Analysis

In [16]:
#trade = "'Will Smith', 'Vladimir Guerrero', 'Emmanuel Clase', 'Bryce Miller', 'Spencer Steer', 'Keibert Ruiz'" #7 #!=1666825
#trade = "'Brandon Drury', 'Michael Kopech', 'Corey Seager', 'Nick Pivetta', 'Sal Frelick', 'Jackson Merrill'"
#trade = "'Oneil Cruz', 'Jordan Westburg', 'Corbin Burnes', 'Randy Arozarena'" #10
trade = "'Julio Rodriguez', 'Fernando Tatis'"
#trade = "'Austin Riley', 'David Bednar', 'Vladimir Guerrero', 'Alex Verdugo', 'Luis Castillo', 'Christopher Morel'"
#trade = "'Garrett Crochet', 'Manny Machado', 'Grayson Rodriguez', 'Daulton Varsho'" #6

pd.read_sql(f"SELECT p.CBSNAME name, sum(AB) AB, sum(R) R, sum(RBI) RBI, sum(HR) HR, sum(SB) SB, \
        round(sum(H)/sum(AB),3) BA, round(sum(outs)/3,2) IP, sum(W) W,sum(SvHld) SvHld,sum(SO) SO, \
        round((sum(Ha)+sum(BBa))/(sum(outs)/3),2) WHIP, round(sum(ER)/(sum(outs)/3)*9,2) ERA \
        FROM stats s \
        INNER JOIN players p On (s.cbsid=p.cbsid) \
        WHERE year=2024 AND s.week >= 14 \
        AND p.cbsid != 1666825 \
        AND p.CBSNAME IN ({trade}) \
        GROUP BY p.CBSNAME", conn)

,name,AB,R,RBI,HR,SB,BA,IP,W,SvHld,SO,WHIP,ERA
0,Fernando Tatis,24.0,3.0,1.0,1.0,1.0,0.167,None,None,None,None,None,None
1,Julio Rodriguez,89.0,14.0,10.0,5.0,4.0,0.292,None,None,None,None,None,None


In [17]:
pd.read_sql(f"SELECT p.CBSNAME, p.cbsid, z.*, s.* \
    FROM vw_player_week_z z INNER JOIN players p On z.cbsid=p.cbsid \
    LEFT JOIN stats s On (z.cbsid=s.cbsid and z.week=s.week and z.year=s.year) \
    WHERE z.year=2024 and z.week>=6 \
    AND p.cbsid != 1666825 \
    AND p.CBSNAME IN ({trade})", conn).groupby('CBSNAME')['z'].sum()

CBSNAME
Fernando Tatis     22.118676
Julio Rodriguez    38.220632
Name: z, dtype: float64

In [8]:
data = pd.read_sql("SELECT s.cbsid, p.CBSNAME name, CASE WHEN AB IS NULL THEN 'p' ELSE 'h' END type, \
        week, H, AB, R, RBI, HR, SB, H/AB BA, outs, (outs/3) IP, \
        W, SvHld, SO, ER, Ha, BBa, ER/(outs/3) ERA, (Ha+BBa)/(outs/3) WHIP \
        FROM stats s \
        INNER JOIN players p On (s.cbsid=p.cbsid) \
        WHERE year=2023", conn)
data

,cbsid,name,type,week,H,AB,R,RBI,HR,SB,BA,outs,IP,W,SvHld,SO,ER,Ha,BBa,ERA,WHIP
0,288897,Miguel Cabrera,h,1,2.0,10.0,0.0,1.0,0.0,0.0,0.200000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,288897,Miguel Cabrera,h,2,2.0,11.0,0.0,1.0,0.0,0.0,0.181818,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,288897,Miguel Cabrera,h,3,3.0,9.0,0.0,1.0,0.0,0.0,0.333333,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,288897,Miguel Cabrera,h,4,0.0,7.0,0.0,0.0,0.0,0.0,0.000000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,288897,Miguel Cabrera,h,5,3.0,16.0,0.0,0.0,0.0,0.0,0.187500,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
22528,29190767,Carson Spiers,p,23,NaN,NaN,NaN,NaN,NaN,NaN,NaN,12.0,4.000000,0.0,0.0,7.0,3.0,5.0,2.0,0.750000,1.750000
22529,29190767,Carson Spiers,p,24,NaN,NaN,NaN,NaN,NaN,NaN,NaN,9.0,3.000000,0.0,0.0,1.0,4.0,5.0,3.0,1.333333,2.666667
22530,29190767,Carson Spiers,p,25,NaN,NaN,NaN,NaN,NaN,NaN,NaN,9.0,3.000000,0.0,0.0,3.0,3.0,7.0,2.0,1.000000,3.000000
22531,29190767,Carson Spiers,p,27,NaN,NaN,NaN,NaN,NaN,NaN,NaN,9.0,3.000000,0.0,1.0,1.0,0.0,1.0,0.0,0.000000,0.333333


In [4]:
def base_sum(row, lgBA):
    return row['name'].max(), row['type'].max(), row['R'].sum(), row['RBI'].sum(), row['HR'].sum(), row['SB'].sum(), row['H'].sum() - (row['AB'].sum()*lgBA)

def pitcher_base(row, lgERA, lgWHIP):
    return row['name'].max(), row['type'].max(), row['W'].sum(), row['SO'].sum(), row['SvHld'].sum(), (((row['ER'].sum()*9) - ((row['outs'].sum()/3)*lgERA))*-1), (((row['Ha'].sum()+row['BBa'].sum()) - ((row['outs'].sum()/3)*lgWHIP))*-1)

In [41]:
def plyr(row, lgBA):
    #(h.H - (h.AB*lgBA)) BA_cnt
    return row['name'].max(), row['R'].sum(), row['RBI'].sum(), row['HR'].sum(), row['SB'].sum(), row['H'].sum() - (row['AB'].sum()*lgBA)

In [9]:
#pd.DataFrame(
data[(data['week']>=wk) & (data['type']=='p')].groupby('cbsid').apply(lambda x: pitcher_base(x, lgERA, lgWHIP))#.reset_index()[0].tolist()

NameError: name 'wk' is not defined

In [42]:
def z_ify(row, m, s):
    return (row-m)/s

In [43]:
def subset(wk):
    lgBA = data[(data['week']>=wk) & (data['type']=='h')]['H'].sum()/data[(data['week']>=wk) & (data['type']=='h')]['AB'].sum()
    lgERA = data[(data['week']>=wk) & (data['type']=='p')]['ER'].sum() / (data[(data['week']>=wk) & (data['type']=='p')]['outs'].sum()/3)*9
    lgWHIP = (data[(data['week']>=wk) & (data['type']=='p')]['Ha'].sum() + data[(data['week']>=wk) & (data['type']=='p')]['BBa'].sum()) / (data[(data['week']>=wk) & (data['type']=='p')]['outs'].sum()/3)

    hitter_means = pd.DataFrame(data[(data['week']>=wk) & (data['type']=='h')].groupby('cbsid').apply(lambda x: base_sum(x, lgBA)).reset_index()[0].tolist(),
                columns=['name', 'type', 'R', 'RBI', 'HR', 'SB', 'BA'])[['R', 'RBI', 'HR', 'SB', 'BA']].mean()
    hitter_stds = pd.DataFrame(data[(data['week']>=wk) & (data['type']=='h')].groupby('cbsid').apply(lambda x: base_sum(x, lgBA)).reset_index()[0].tolist(),
                columns=['name', 'type', 'R', 'RBI', 'HR', 'SB', 'BA'])[['R', 'RBI', 'HR', 'SB', 'BA']].std()
    pitcher_means = pd.DataFrame(data[(data['week']>=wk) & (data['type']=='p')].groupby('cbsid').apply(lambda x: pitcher_base(x, lgERA, lgWHIP)).reset_index()[0].tolist(),
                columns=['name', 'type', 'W', 'SO', 'SvHld', 'ERA', 'WHIP'])[['W', 'SO', 'SvHld', 'ERA', 'WHIP']].mean()
    pitcher_stds = pd.DataFrame(data[(data['week']>=wk) & (data['type']=='p')].groupby('cbsid').apply(lambda x: pitcher_base(x, lgERA, lgWHIP)).reset_index()[0].tolist(),
                columns=['name', 'type', 'W', 'SO', 'SvHld', 'ERA', 'WHIP'])[['W', 'SO', 'SvHld', 'ERA', 'WHIP']].std()

    a = pd.DataFrame(data[(data['type']=='h') & (data['week']>=wk)].groupby('cbsid').apply(lambda x: plyr(x, lgBA)).tolist(),
        columns=['name','R', 'RBI', 'HR', 'SB', 'BA'])

    for stat in ['R', 'RBI', 'HR', 'SB', 'BA']:
        a['v'+stat] = a[stat].apply(lambda x: z_ify(x,hitter_means[stat], hitter_stds[stat]))

    a['z'] = a['vR'] + a['vRBI'] + a['vHR'] + a['vSB'] + a['vBA']

    b = pd.DataFrame(data[(data['week']>=wk) & (data['type']=='p')].groupby('cbsid').apply(lambda x: pitcher_base(x, lgERA, lgWHIP)).tolist(),
        columns=['name', 'type', 'W', 'SO', 'SvHld', 'ERA', 'WHIP'])

    for stat in ['W', 'SO', 'SvHld', 'ERA', 'WHIP']:
        b['v'+stat] = b[stat].apply(lambda x: z_ify(x, pitcher_means[stat], pitcher_stds[stat]))

    b['z'] = b['vW'] + b['vSO'] + b['vSvHld'] + b['vERA'] + b['vWHIP']

    c = pd.concat([a,b])
    return c

In [399]:
c = subset(16)
c = c.merge(data.groupby('name')['cbsid'].max(), left_on='name', right_index=True, how='inner')
c[c['cbsid'].isin(trades)].sort_values('z', ascending=False)

,name,R,RBI,HR,SB,BA,vR,vRBI,vHR,vSB,vBA,z,type,W,SO,SvHld,ERA,WHIP,vW,vSO,vSvHld,vERA,vWHIP
228,Sandy Alcantara,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,5.076208,p,4.0,57.0,0.0,83.422806,11.638745,1.349172,1.291423,-0.562281,1.628698,1.369197
454,Alec Bohm,37.0,40.0,11.0,1.0,5.097259,1.363873,1.635917,1.246894,-0.383403,0.845741,4.709022,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
296,Jazz Chisholm,30.0,30.0,10.0,8.0,0.988821,0.870996,0.927835,1.046632,1.183993,0.164066,4.193523,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
473,Tony Gonsolin,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,-1.976437,p,3.0,28.0,0.0,-97.057737,-3.461104,0.807223,0.080688,-0.562281,-1.894898,-0.407169


In [385]:
c.sort_values('z', ascending=False).head(15)

,name,R,RBI,HR,SB,BA,vR,vRBI,vHR,vSB,vBA,z,type,W,SO,SvHld,ERA,WHIP,vW,vSO,vSvHld,vERA,vWHIP
248,Ronald Acuna,93.0,69.0,28.0,45.0,35.269713,3.768074,2.554220,3.436338,7.063834,4.971064,21.793531,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
90,Blake Snell,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,15.536564,p,12.0,153.0,0.0,341.238407,23.876395,4.318303,3.799754,-0.538323,5.623264,2.333567
117,Matt Olson,81.0,94.0,36.0,0.0,25.756937,3.128287,3.895425,4.684604,-0.586923,3.630293,14.751686,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
541,Kyle Bradish,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,14.466155,p,10.0,117.0,0.0,253.638820,48.009336,3.465658,2.666902,-0.538323,4.179711,4.692207
55,Gerrit Cole,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,14.439491,p,8.0,132.0,0.0,233.153785,55.084883,2.613014,3.138924,-0.538323,3.842138,5.383738
514,Bobby Witt,60.0,68.0,20.0,29.0,20.017584,2.008660,2.500572,2.188071,4.343565,2.821364,13.862232,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
362,Julio Rodriguez,63.0,67.0,20.0,25.0,18.031779,2.168607,2.446924,2.188071,3.663498,2.541476,13.008576,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
147,Mookie Betts,73.0,66.0,22.0,10.0,29.989193,2.701763,2.393276,2.500138,1.113246,4.226805,12.935227,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
223,Kyle Tucker,72.0,74.0,21.0,20.0,15.493452,2.648447,2.822461,2.344104,2.813414,2.183713,12.812140,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
13,Freddie Freeman,74.0,58.0,16.0,15.0,28.765455,2.755078,1.964090,1.563938,1.963330,4.054326,12.300762,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [51]:
trades = {"9 Grand Kids":[
        {'id':1, 'week':7, 'partner':'Wiscompton Wu-Tang', 'received':[26615410], 'traded':[2167484]}
    ],
 "Brewbirds":[
     {'id':2, 'week':7, 'partner':'Wiscompton Wu-Tang', 'received':[1600680], 'traded':[2835054]}
     ],
 'Charmer':[
     {'id':3, 'week':5, 'partner':'Wiscompton Wu-Tang', 'received':[2942981], 'traded':[2066704]},
     {'id':4, 'week':19, 'partner':'Lima Time!', 'received':[2000027], 'traded':[2911279]},
     ],
 'Harveys Wallbangers':[
     {'id':5, 'week':16, 'partner':'Wiscompton Wu-Tang', 'received':[2942960,2211794], 'traded':[2507358,2915614]},
     ],
 'Lil Trump & the Ivanabees':[
     {'id':6, 'week':21, 'partner':'Wiscompton Wu-Tang', 'received':[2507358,2824831], 'traded':[1893753]},
     ],
 'Lima Time!':[
     {'id':4, 'week':19, 'partner':'Charmer', 'received':[2911279], 'traded':[2000027]},
     {'id':7, 'week':7, 'partner':'Ugly Spuds', 'received':[2184351,1937347], 'traded':[2942960,530362]},
     {'id':8, 'week':12, 'partner':'Ugly Spuds', 'received':[2041873], 'traded':[2001082]},
     {'id':9, 'week':22, 'partner':'Wiscompton Wu-Tang', 'received':[2942984,2821213,3117421], 'traded':[1937347,3095687]},
     ],
 'Roid Ragers':[
     {'id':10, 'week':13, 'partner':'Wiscompton Wu-Tang', 'received':[2167484,2837306], 'traded':[3117421,1894628]},
     ],
 'Ugly Spuds':[
     {'id':7, 'week':7, 'partner':'Lima Time!', 'received':[2942960,530362], 'traded':[2184351,1937347]},
     {'id':8, 'week':12, 'partner':'Lima Time!', 'received':[2001082], 'traded':[2041873]},
     {'id':11, 'week':9, 'partner':'Wiscompton Wu-Tang', 'received':[26911810], 'traded':[2942960]},
     {'id':12, 'week':13, 'partner':'Wiscompton Wu-Tang', 'received':[2006996,1967936,2918658], 'traded':[530362,3152036,2167482]},
     {'id':13, 'week':22, 'partner':'Wiscompton Wu-Tang', 'received':[2447486], 'traded':[26911810]},
     ],
 'Wiscompton Wu-Tang':[
     {'id':1, 'week':7, 'partner':'9 Grand Kids', 'received':[2167484], 'traded':[26615410]},
     {'id':2, 'week':7, 'partner':'Brewbirds', 'received':[2835054], 'traded':[1600680]},
     {'id':3, 'week':5, 'partner':'Charmer', 'received':[2066704], 'traded':[2942981]},
     {'id':5, 'week':16, 'partner':'Harveys Wallbangers', 'received':[2507358,2915614], 'traded':[2942960,2211794]},
     {'id':6, 'week':21, 'partner':'Lil Trump & the Ivanabees', 'received':[1893753], 'traded':[2507358,2824831]},
     {'id':9, 'week':22, 'partner':'Lima Time!', 'received':[1937347,3095687], 'traded':[2942984,2821213,3117421]},
     {'id':10, 'week':13, 'partner':'Roid Ragers', 'received':[3117421,1894628], 'traded':[2167484,2837306]},
     {'id':11, 'week':9, 'partner':'Ugly Spuds', 'received':[2942960], 'traded':[26911810]},
     {'id':12, 'week':13, 'partner':'Ugly Spuds', 'received':[530362,3152036,2167482], 'traded':[2006996,1967936,2918658]},
     {'id':13, 'week':22, 'partner':'Ugly Spuds', 'received':[26911810], 'traded':[2447486]},
     ],
}

In [47]:
c = subset(trades['9 Grand Kids'][0]['week'])
c = c.merge(data.groupby('name')['cbsid'].max(), left_on='name', right_index=True, how='inner')

In [49]:
c[c['cbsid'].isin(trades['9 Grand Kids'][0]['received']+trades['9 Grand Kids'][0]['traded'])]

,name,R,RBI,HR,SB,BA,vR,vRBI,vHR,vSB,vBA,z,type,W,SO,SvHld,ERA,WHIP,vW,vSO,vSvHld,vERA,vWHIP,cbsid
210,Jorge Mateo,34.0,15.0,1.0,20.0,-14.164928,0.250664,-0.498646,-0.800524,2.154163,-1.683606,-0.577948,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2167484
713,Bryce Elder,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2.596083,p,9.0,92.0,0.0,-18.396638,-2.13575,2.310718,1.278506,-0.525044,-0.279548,-0.188549,26615410


In [95]:
for tm in trades.keys():
    for n in range(len(trades[tm])):
        c = subset(trades[tm][n]['week'])
        c = c.merge(data.groupby('name')['cbsid'].max(), left_on='name', right_index=True, how='inner')
        trades[tm][n]['received_names'] = c[c['cbsid'].isin(trades[tm][n]['received'])]['name'].tolist()
        trades[tm][n]['traded_names'] = c[c['cbsid'].isin(trades[tm][n]['traded'])]['name'].tolist()
        #trades[tm][n]['received_value'] = c[c['cbsid'].isin(trades[tm][n]['received'])].z.sum()
        trades[tm][n]['received_stats'] = c[c['cbsid'].isin(trades[tm][n]['received'])][['z', 'R', 'RBI', 'HR', 'SB', 'BA', 'W', 'SO', 'SvHld', 'ERA', 'WHIP']].sum().to_dict()
        #trades[tm][n]['traded_value'] = c[c['cbsid'].isin(trades[tm][n]['traded'])].z.sum()
        trades[tm][n]['traded_stats'] = c[c['cbsid'].isin(trades[tm][n]['traded'])][['z', 'R', 'RBI', 'HR', 'SB', 'BA', 'W', 'SO', 'SvHld', 'ERA', 'WHIP']].sum().to_dict()

In [96]:
import json
with open('trades_2023.json', 'w') as f:
    json.dump(trades, f)

In [65]:
c[c['cbsid'].isin(trades[tm][n]['traded'])][['R', 'RBI', 'HR', 'SB', 'BA', 'W', 'SO', 'SvHld', 'ERA', 'WHIP']].sum().to_dict()

{'R': 27.0,
 'RBI': 26.0,
 'HR': 8.0,
 'SB': 12.0,
 'BA': 10.790127970749545,
 'W': 0.0,
 'SO': 0.0,
 'SvHld': 0.0,
 'ERA': 0.0,
 'WHIP': 0.0}

In [82]:
v_rec, v_tra = 0,0
for n in trades['Ugly Spuds']:
    v_rec += n['received_stats']['z']
    v_tra += n['traded_stats']['z']
v_rec, v_tra

(47.000077379136336, 33.31801706461731)

In [116]:
for tm in trades.keys():
    diff = 0
    if trades.get(tm):
        for j in trades[tm]:
            diff += j['received_value'] - j['traded_value']
    print(tm, diff)

9 Grand Kids 3.1740309133103715
Brewbirds 7.221137599654294
Charmer -10.604998172488312
Harveys Wallbangers 7.568143709207094
Lil Trump & the Ivanabees -4.7354431931686145
Lima Time! 29.513433119451456
Roid Ragers -1.1711160936461544
Ugly Spuds 13.682060314519035
Wiscompton Wu-Tang -44.64724819683917


In [133]:
pd.concat([
    pd.DataFrame(data.groupby(['week', 'type']).apply(lambda x: new(x)).unstack().reset_index()['h'].tolist(),
            columns=['R_mean', 'R_std', 'RBI_mean', 'RBI_std', 'HR_mean', 'HR_std', 'SB_mean', 'SB_std', 'BA', 
                     'W_mean', 'W_std', 'SO_mean', 'SO_std', 'SvHld_mean', 'SvHld_std', 'ERA', 'WHIP']),
    pd.DataFrame(data.groupby(['week', 'type']).apply(lambda x: new(x)).unstack().reset_index()['p'].tolist(),
            columns=['R_mean', 'R_std', 'RBI_mean', 'RBI_std', 'HR_mean', 'HR_std', 'SB_mean', 'SB_std', 'BA', 
                     'W_mean', 'W_std', 'SO_mean', 'SO_std', 'SvHld_mean', 'SvHld_std', 'ERA', 'WHIP'])
    ]
)
#.agg({'H':'sum', 'AB':'sum', 'R':'sum', 'RBI':'sum', 'HR':'sum', 'SB':'sum'})

,R_mean,R_std,RBI_mean,RBI_std,HR_mean,HR_std,SB_mean,SB_std,BA,W_mean,W_std,SO_mean,SO_std,SvHld_mean,SvHld_std,ERA,WHIP
0,1.113757,1.223237,1.044974,1.467812,0.267196,0.545085,0.185185,0.522788,0.245487,0.000000,NaN,10.000000,NaN,0.000000,NaN,0.000000,0.833333
1,2.243781,1.946997,2.166667,2.197161,0.567164,0.836640,0.313433,0.659682,0.251371,1.000000,NaN,8.000000,NaN,0.000000,NaN,1.500000,1.166667
2,2.168317,1.850863,2.086634,2.242145,0.509901,0.846772,0.373762,0.775816,0.247207,1.000000,NaN,6.000000,NaN,0.000000,NaN,0.000000,0.857143
3,2.080605,1.823957,2.007557,2.114749,0.539043,0.814398,0.272040,0.612619,0.234632,1.000000,NaN,14.000000,NaN,0.000000,NaN,1.000000,0.555556
4,2.189526,1.748997,2.109726,1.975584,0.526185,0.803080,0.366584,0.817178,0.254473,1.000000,NaN,8.000000,NaN,0.000000,NaN,7.500000,0.833333
5,2.190594,1.883930,2.096535,2.089341,0.551980,0.809234,0.336634,0.739220,0.251846,0.000000,NaN,13.000000,NaN,0.000000,NaN,7.200000,1.200000
6,1.891089,1.712027,1.811881,1.994846,0.504950,0.776501,0.299505,0.695180,0.242117,0.000000,NaN,7.000000,NaN,0.000000,NaN,3.857143,1.142857
7,2.140394,1.859907,2.064039,2.106038,0.556650,0.875168,0.312808,0.687226,0.251466,1.000000,NaN,14.000000,NaN,0.000000,NaN,4.153846,0.846154
8,2.275689,1.998539,2.200501,2.141924,0.601504,0.856015,0.358396,0.739668,0.248902,0.000000,NaN,10.000000,NaN,0.000000,NaN,1.500000,1.500000
9,1.892421,1.572798,1.801956,1.977173,0.447433,0.732843,0.371638,0.818761,0.243656,0.000000,NaN,6.000000,NaN,0.000000,NaN,7.500000,1.666667


In [153]:
pd.DataFrame(index=['h'], columns=['R_mean', 'R_std', 'RBI_mean', 'RBI_std', 'HR_mean', 'HR_std', 
                                   'SB_mean', 'SB_std', 'BA','W_mean', 'W_std', 'SO_mean', 'SO_std', 
                                   'SvHld_mean', 'SvHld_std', 'ERA', 'WHIP'], 
             data=data[data['week']>=12].groupby('type').apply(lambda x: new(x))['h'])

ValueError: Shape of passed values is (17, 1), indices imply (1, 17)

In [159]:
pd.concat([
    pd.DataFrame([data[data['week']>=12].groupby('type').apply(lambda x: new(x))['h']],
            index=['h'], 
            columns=['R_mean', 'R_std', 'RBI_mean', 'RBI_std', 'HR_mean', 'HR_std', 
                                   'SB_mean', 'SB_std', 'BA','W_mean', 'W_std', 'SO_mean', 'SO_std', 
                                   'SvHld_mean', 'SvHld_std', 'ERA', 'WHIP']),
    pd.DataFrame([data[data['week']>=12].groupby('type').apply(lambda x: new(x))['h']],
            index=['p'], 
            columns=['R_mean', 'R_std', 'RBI_mean', 'RBI_std', 'HR_mean', 'HR_std', 
                                   'SB_mean', 'SB_std', 'BA','W_mean', 'W_std', 'SO_mean', 'SO_std', 
                                   'SvHld_mean', 'SvHld_std', 'ERA', 'WHIP'])
    ]
)

,R_mean,R_std,RBI_mean,RBI_std,HR_mean,HR_std,SB_mean,SB_std,BA,W_mean,W_std,SO_mean,SO_std,SvHld_mean,SvHld_std,ERA,WHIP
h,2.05733,1.842109,1.971107,2.071443,0.550791,0.851305,0.318127,0.713208,0.24929,0.5,0.527046,6.5,3.24037,0.0,0.0,2.892857,1.089286
p,2.05733,1.842109,1.971107,2.071443,0.550791,0.851305,0.318127,0.713208,0.24929,0.5,0.527046,6.5,3.24037,0.0,0.0,2.892857,1.089286


In [125]:
def new(row):
    return row['R'].mean(), row['R'].std(), row['RBI'].mean(), row['RBI'].std(), row['HR'].mean(), \
            row['HR'].std(), row['SB'].mean(), row['SB'].std(), row['H'].mean()/row['AB'].mean(), \
            row['W'].mean(), row['W'].std(), row['SO'].mean(), row['SO'].std(), row['SvHld'].mean(), \
            row['SvHld'].std(), row['ER'].mean()/(row['outs'].mean()/3)*9, (row['Ha'].mean()+row['BBa'].mean())/(row['outs'].mean()/3)

#unpack tuple into columns in a dataframe
#https://stackoverflow.com/questions/29550414/how-can-i-split-a-column-of-tuples-in-a-pandas-dataframe

### Weekly Optimization Score

In [93]:
a = pd.read_sql("SELECT distinct p.CBSNAME name, r.owner_id, r.decision, z.* \
        FROM vw_player_week_z z \
        LEFT JOIN players p on p.cbsid=z.cbsid \
        INNER JOIN roster r on (r.cbsid=z.cbsid and r.year=z.year and r.week=z.week) \
        WHERE z.year=2024 AND z.week=r.week \
        ORDER BY z.year, z desc", conn)

In [94]:
a[(a['owner_id']==38) & (a['week']>=14)]

,name,owner_id,decision,year,week,cbsid,zR,zRBI,zHR,zSB,zBA,zHit,zW,zSO,zSvHld,zERA,zWHIP,zPitch,z,SvHld
154,Garrett Crochet,38,start,2024,15,2836763,NaN,NaN,NaN,NaN,NaN,NaN,-0.499971,4.118942,-0.443059,1.965243,2.193019,7.334174,7.334174,0.0
183,Nico Hoerner,38,start,2024,15,2942984,0.579594,0.197083,0.600527,3.943986,1.667123,6.988314,NaN,NaN,NaN,NaN,NaN,NaN,6.988314,NaN
191,Justin Steele,38,start,2024,16,2211224,NaN,NaN,NaN,NaN,NaN,NaN,1.722615,1.077051,-0.521162,1.769408,2.856616,6.904528,6.904528,0.0
211,David Hamilton,38,start,2024,14,3158363,0.825752,0.481227,0.121173,4.532765,0.767440,6.728356,NaN,NaN,NaN,NaN,NaN,NaN,6.728356,NaN
486,Bryan Abreu,38,start,2024,14,3055522,NaN,NaN,NaN,NaN,NaN,NaN,-0.489045,-0.167608,3.958615,0.747212,0.986794,5.035968,5.035968,3.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5094,Andres Munoz,38,start,2024,15,2910638,NaN,NaN,NaN,NaN,NaN,NaN,-0.499971,-0.869135,-0.443059,-0.195797,-0.868113,-2.876075,-2.876075,0.0
5123,Bryan Abreu,38,start,2024,15,3055522,NaN,NaN,NaN,NaN,NaN,NaN,-0.499971,-0.557380,-0.443059,-0.784401,-0.651085,-2.935896,-2.935896,0.0
5179,Reed Garrett,38,start,2024,15,2739980,NaN,NaN,NaN,NaN,NaN,NaN,-0.499971,-0.557380,-0.443059,-0.470156,-1.146337,-3.116903,-3.116903,0.0
5392,Yusei Kikuchi,38,start,2024,14,3073522,NaN,NaN,NaN,NaN,NaN,NaN,-0.489045,2.259025,-0.481736,-3.252048,-2.934173,-4.897977,-4.897977,0.0


### Standings

In [36]:
opening_week = int(datetime.strftime(pd.Timestamp('2024-03-28'), '%W')) - 1
cur_week = int(datetime.strftime(datetime.now(), '%W'))
period = cur_week - opening_week
current_period = period + 1
opening_week, cur_week, period, current_period

(12, 29, 17, 18)

In [51]:
st = pd.read_sql(f"SELECT p.CBSNAME Player, s.*, r.owner_id, o.owner, r.decision, r.pos \
        FROM roster r \
        INNER JOIN stats s On (r.cbsid=s.cbsid and r.week=s.week and r.year=s.year) \
        INNER JOIN owners o On (r.owner_id=o.owner_id) \
        LEFT JOIN players p On (r.cbsid=p.cbsid) \
        WHERE r.year=2024 AND decision='start'", conn).drop_duplicates()
st.shape

(4505, 22)

In [52]:
st[st['owner']=='Lima Time!'][['HR', 'SB', 'R','RBI', 'SvHld', 'ER', 'outs']].sum()

HR        169.0
SB         90.0
R         617.0
RBI       653.0
SvHld      52.0
ER        290.0
outs     2452.0
dtype: float64

In [53]:
# Group by owner and sum counting stats
counting_stats = ['R', 'RBI', 'HR', 'SB', 'W', 'SO', 'SvHld', 'H', 'AB', 'Ha', 'BBa', 'ER', 'IP']

# Columns to rank in ascending order (better to have more)
ascending_columns = ['R', 'RBI', 'HR', 'SB', 'W', 'SO', 'SvHld', 'BA']

# Columns to rank in descending order (better to have less)
descending_columns = ['ERA', 'WHIP']

grouped = st.groupby('owner')[counting_stats].sum()

# Calculate rate stats
rate_stats = st.groupby('owner').apply(lambda x: pd.Series({
    'BA': x['H'].sum() / x['AB'].sum(),
    'ERA': x['ER'].sum() / (x['outs'].sum() / 3) * 9,
    'WHIP': (x['Ha'].sum() + x['BBa'].sum()) / (x['outs'].sum() / 3)
}))

standings = pd.concat([grouped, rate_stats], axis=1)

# Rank each column with appropriate ascending argument
ranked_data = {}
for col in ascending_columns:
    ranked_data[col + '_rank'] = standings[col].rank(ascending=True)

for col in descending_columns:
    ranked_data[col + '_rank'] = standings[col].rank(ascending=False)

# Create a DataFrame for ranked data
ranked_df = pd.DataFrame(ranked_data, index=standings.index)
ranked_df

,R_rank,RBI_rank,HR_rank,SB_rank,W_rank,SO_rank,SvHld_rank,BA_rank,ERA_rank,WHIP_rank
owner,,,,,,,,,,
9 Grand Kids,10.0,10.0,8.0,5.0,11.0,11.0,5.5,12.0,5.0,2.0
Brewbirds,9.0,7.0,10.0,7.0,12.0,9.0,10.0,5.0,7.0,6.0
Charmer,11.0,9.0,12.0,11.0,2.0,2.0,12.0,10.0,11.0,12.0
Dirty Birds,1.0,2.0,1.0,1.0,5.0,1.0,11.0,1.0,3.0,4.0
Harveys Wallbangers,12.0,5.0,5.5,8.0,10.0,4.0,9.0,8.0,9.0,10.0
Lima Time!,5.0,12.0,7.0,4.0,2.0,12.0,5.5,9.0,12.0,9.0
Mom's Cookin,3.0,3.0,5.5,3.0,8.0,3.0,2.0,3.0,10.0,8.0
Roid Ragers,7.0,8.0,11.0,2.0,2.0,6.0,4.0,7.0,6.0,7.0
Trouble with the Curve,8.0,6.0,4.0,6.0,7.0,7.0,7.5,11.0,2.0,3.0


In [54]:
standings#.loc['Lima Time!']

,R,RBI,HR,SB,W,SO,SvHld,H,AB,Ha,BBa,ER,IP,BA,ERA,WHIP
owner,,,,,,,,,,,,,,,,
9 Grand Kids,643.0,626.0,170.0,91.0,55.0,838.0,52.0,1236.0,4526.0,795.0,250.0,372.0,850.66,0.273089,3.935737,1.228448
Brewbirds,640.0,603.0,175.0,98.0,59.0,830.0,63.0,1128.0,4582.0,713.0,231.0,312.0,804.00,0.246181,3.492537,1.174129
Charmer,652.0,617.0,185.0,149.0,46.0,640.0,89.0,1199.0,4685.0,584.0,192.0,253.0,689.00,0.255923,3.304790,1.126270
Dirty Birds,525.0,528.0,133.0,67.0,49.0,602.0,85.0,1020.0,4375.0,600.0,184.0,295.0,645.35,0.233143,4.114153,1.214876
Harveys Wallbangers,661.0,585.0,167.0,109.0,54.0,755.0,55.0,1166.0,4599.0,617.0,233.0,282.0,745.31,0.253533,3.405188,1.140429
Lima Time!,617.0,653.0,169.0,90.0,46.0,884.0,52.0,1169.0,4594.0,696.0,238.0,290.0,817.32,0.254462,3.193312,1.142741
Mom's Cookin,589.0,537.0,167.0,73.0,52.0,729.0,38.0,1022.0,4263.0,672.0,239.0,289.0,783.31,0.239737,3.320426,1.162979
Roid Ragers,629.0,606.0,180.0,71.0,46.0,765.0,43.0,1130.0,4516.0,620.0,241.0,305.0,734.67,0.250221,3.736388,1.171960
Trouble with the Curve,635.0,600.0,158.0,95.0,51.0,797.0,54.0,1140.0,4392.0,734.0,268.0,376.0,818.69,0.259563,4.133550,1.223941


In [311]:
proj.loc[(proj['all_pos']==0) & (proj['pos'].isin(['SP', 'RP', 'P'])), 'all_pos'] = 'P'
proj.loc[(proj['all_pos']==0) & (proj['pos'].isin(['C', '1B', '2B', '3B', 'SS', 'OF', 'MI', 'CI', 'DH'])), 'all_pos'] = proj.loc[proj['pos'].isin(['C', '1B', '2B', '3B', 'SS', 'OF', 'MI', 'CI', 'DH']), 'pos']+',DH'

In [312]:
temp = {}

for tm in standings.index:
    print('\n',tm)
    opt = optimize(tm, proj)
    s = proj[proj['CBSNAME'].isin(opt.hitter_optimized_lineup+opt.pitcher_optimized_lineup)][['R', 'RBI', 'HR', 'SB', 'H', 'AB', 'W', 'SO', 'SvHld', 'ER', 'BBa', 'Ha', 'IP']].sum()
    #s['SvHld'] = s['Sv+Hld']
    s['BA'] = s['H']/s['AB']
    s['ERA'] = s['ER']/s['IP']*9
    s['WHIP'] = (s['Ha']+s['BBa'])/s['IP']
    temp[tm] = s[['R', 'RBI', 'HR', 'SB', 'W', 'SO', 'SvHld', 'BA', 'ERA', 'WHIP', 'H', 'AB', 'Ha', 'BBa', 'ER', 'IP']]

22.511008322924578 ['George Kirby', 'Joe Ryan', 'Kevin Gausman', 'Carlos Rodon', 'Framber Valdez', 'Camilo Doval', 'Peter Fairbanks', 'Trevor Megill', 'Aaron Civale']
23.922253385927405 ['Salvador Perez', 'Jeimer Candelario', 'Ozzie Albies', 'Carlos Correa', 'Manny Machado', 'Trea Turner', "Ke'Bryan Hayes", 'Steven Kwan', 'Ian Happ', 'Lourdes Gurriel', 'Brandon Marsh', 'Wilyer Abreu', 'Ezequiel Tovar', 'Lars Nootbaar']
18.149110344120217 ['Chris Sale', 'Sonny Gray', 'Logan Webb', 'Ryan Helsley', 'David Bednar', 'Yoshinobu Yamamoto', 'Kenley Jansen', 'Jacob deGrom', 'Carlos Estevez']
34.30652287146305 ['Willson Contreras', 'Vladimir Guerrero', 'Andres Gimenez', 'Francisco Lindor', 'Jose Ramirez', 'Gleyber Torres', 'Austin Riley', 'Yordan Alvarez', 'Alec Burleson', 'James Wood', 'Jake Fraley', 'Will Benson', 'J.D. Martinez', 'Colt Keith']
19.25437755201511 ['Corbin Burnes', 'Logan Gilbert', 'Luis Castillo', 'Emmanuel Clase', 'Taj Bradley', 'Evan Phillips', 'Cristopher Sanchez', 'Jeff Hof

In [313]:
final_proj = pd.DataFrame([temp[i] for i in list(temp.keys())], index=list(temp.keys()))+standings
final_proj['BA'] = final_proj['H']/final_proj['AB']
final_proj['ERA'] = final_proj['ER']/final_proj['IP']*9
final_proj['WHIP'] = (final_proj['Ha']+final_proj['BBa'])/final_proj['IP']

In [314]:
final_standings = final_proj[['R', 'RBI', 'HR','SB', 'BA', 'W', 'SO', 'SvHld', 'ERA', 'WHIP']]
for stat in counting_stats:
    try:
        final_standings[stat+'_rank'] = final_standings[stat].rank()
    except:
        pass
for stat in rate_stats:
    try:
        final_standings[stat+'_rank'] = final_standings[stat].rank(ascending=False)
    except:
        pass
final_standings.iloc[:,10:].sum(axis=1).sort_values(ascending=False)

C:\ProgramData\Anaconda3\envs\Python_37\lib\site-packages\ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  after removing the cwd from sys.path.


Brewbirds                 92.0
Charmer                   84.0
Lima Time!                77.0
9 Grand Kids              73.0
Harveys Wallbangers       70.0
Ugly Spuds                66.0
Roid Ragers               61.0
Wiscompton Wu-Tang        61.0
Trouble with the Curve    60.0
Young Guns                50.0
Mom's Cookin              48.0
Dirty Birds               38.0
dtype: float64

In [315]:
final_standings['R'].rank()

9 Grand Kids               9.0
Brewbirds                 12.0
Charmer                   11.0
Dirty Birds                1.0
Harveys Wallbangers       10.0
Lima Time!                 7.0
Mom's Cookin               2.0
Roid Ragers                8.0
Trouble with the Curve     6.0
Ugly Spuds                 3.0
Wiscompton Wu-Tang         5.0
Young Guns                 4.0
Name: R, dtype: float64

In [344]:
stat = 'R'
pt = pd.pivot_table(st, index='owner', columns='week', values=stat, aggfunc='sum')
pt['total'] = pt.sum(axis=1)
pt['mean'] = pt.iloc[:,1:period].mean(axis=1)
pt['std'] = pt.iloc[:,1:period].std(axis=1)
pt['min'] = pt.loc[:,'mean']-(2*pt.loc[:, 'std'])
pt['max'] = pt.loc[:,'mean']+(2*pt.loc[:, 'std'])
pt

week,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,total,mean,std,min,max
owner,,,,,,,,,,,,,,,,,,,,,
9 Grand Kids,0.0,40.0,47.0,45.0,45.0,39.0,30.0,32.0,30.0,44.0,31.0,45.0,43.0,41.0,38.0,47.0,597.0,39.800000,6.258708,27.282584,52.317416
Brewbirds,0.0,26.0,41.0,49.0,40.0,37.0,33.0,38.0,45.0,39.0,39.0,47.0,34.0,36.0,42.0,51.0,597.0,39.800000,6.504943,26.790114,52.809886
Charmer,8.0,29.0,42.0,39.0,40.0,36.0,31.0,41.0,42.0,51.0,39.0,43.0,49.0,48.0,38.0,42.0,618.0,40.666667,6.007931,28.650804,52.682529
Dirty Birds,0.0,24.0,38.0,32.0,40.0,33.0,31.0,29.0,31.0,33.0,31.0,35.0,26.0,33.0,34.0,45.0,495.0,33.000000,5.223573,22.552854,43.447146
Harveys Wallbangers,4.0,35.0,41.0,61.0,36.0,41.0,39.0,36.0,29.0,43.0,40.0,40.0,48.0,43.0,38.0,41.0,615.0,40.733333,7.075780,26.581773,54.884894
Lima Time!,5.0,22.0,35.0,56.0,43.0,38.0,40.0,36.0,31.0,33.0,45.0,37.0,33.0,38.0,39.0,37.0,568.0,37.533333,7.414914,22.703505,52.363162
Mom's Cookin,2.0,22.0,30.0,30.0,31.0,36.0,38.0,34.0,47.0,36.0,43.0,35.0,44.0,55.0,47.0,41.0,571.0,37.933333,8.396144,21.141045,54.725622
Roid Ragers,0.0,25.0,47.0,28.0,41.0,45.0,53.0,34.0,58.0,37.0,31.0,36.0,32.0,34.0,51.0,47.0,599.0,39.933333,9.815925,20.301484,59.565183
Trouble with the Curve,0.0,32.0,37.0,50.0,33.0,40.0,34.0,35.0,41.0,43.0,52.0,40.0,38.0,38.0,45.0,40.0,598.0,39.866667,5.792442,28.281783,51.451550


In [339]:
tm = 'Lima Time!'
optimized = optimize(tm, df)
optimized2 = optimize(tm, df2)

20.082569810998038 ['Garrett Crochet', 'Shota Imanaga', 'Cole Ragans', 'Edwin Diaz', 'Yusei Kikuchi', 'Hunter Brown', 'Andres Munoz', 'Justin Steele', 'Bryan Abreu']
34.974529489409164 ['Yainer Diaz', 'Josh Naylor', 'Ketel Marte', 'Nico Hoerner', 'Alex Bregman', 'Xander Bogaerts', 'Ryan Mountcastle', 'Kyle Tucker', 'Bryan Reynolds', 'Teoscar Hernandez', 'Brandon Nimmo', 'TJ Friedl', 'Marcell Ozuna', 'Thairo Estrada']
20.082569810998038 ['Garrett Crochet', 'Shota Imanaga', 'Cole Ragans', 'Edwin Diaz', 'Yusei Kikuchi', 'Hunter Brown', 'Andres Munoz', 'Justin Steele', 'Bryan Abreu']
35.98101304105987 ['Yainer Diaz', 'Josh Naylor', 'Jose Altuve', 'Nico Hoerner', 'Alex Bregman', 'Ketel Marte', 'Ryan Mountcastle', 'Kyle Tucker', 'Bryan Reynolds', 'Brandon Nimmo', 'TJ Friedl', 'Jacob Young', 'Marcell Ozuna', 'Xander Bogaerts']


In [345]:
rbi = df[df['Player'].isin(optimized.hitter_optimized_lineup)][['HR', 'SB', 'R', 'RBI']].sum()[stat]/(29-period)
rbi2 = df2[df2['Player'].isin(optimized2.hitter_optimized_lineup)][['HR', 'SB', 'R', 'RBI']].sum()[stat]/(29-period)

fig = go.Figure()

fig.add_trace(
    go.Scatter(name='max',
        x=np.arange(period, 29),
        y=[pt.loc[tm,'total']+(pt.loc[tm, 'max']*i) for i in range(29-period)],
        marker=dict(color='lightgray'),
        #fill='tonexty',
    )
)
fig.add_trace(
    go.Scatter(name='min',
        x=np.arange(period, 29),
        y=[pt.loc[tm,'total']+(pt.loc[tm, 'min']*i) for i in range(29-period)],
        marker=dict(color='lightgray'),
        fill='tonexty',
    )
)
fig.add_trace(
    go.Scatter(name='actual',
        x=np.arange(1,current_period),
        y=pt.loc[tm,1:period].cumsum().values
    )
)
fig.add_trace(
    go.Scatter(name='projection',
        x=np.arange(current_period-1,29),
        y=[pt.loc[tm, 'total']]+[(pt.loc[tm, 'total']+(i*rbi)) for i in range(1,29-period)],
        marker=dict(color='gray')
    )
)
fig.add_trace(
    go.Scatter(name='trade projection',
        x=np.arange(current_period-1,29),
        y=[pt.loc[tm, 'total']]+[(pt.loc[tm, 'total']+(i*rbi2)) for i in range(1,29-period)],
        marker=dict(color='gray'),
        line=dict(dash='dot')
    )
)
fig.update_layout(
    title=tm+' '+stat,
    xaxis = dict(dtick=1)
)
fig.show()

### Prospective Trade Analysis

In [76]:
def load(period):
    df = pd.read_csv(f'data\\2024-period-{period}-ros-projections.csv')
    #df.loc[df['Player'].isin(['Nico Hoerner', 'Bryson Stott']), 'all_pos'] = "['2B', 'SS', 'MI', 'DH']"
    #df.loc[df['Player'].isin(['Isaac Paredes', ]), 'all_pos'] = "['2B', '3B', '1B', 'MI', 'CI']"
    #df.loc[df['Player'].isin(['Max Muncy', ]), 'all_pos'] = "['2B', '3B', 'MI', 'CI']"
    return df

In [8]:
def optimize_team(tm, df):
    w = ol.Optimized_Lineups(tm, df[df['Owner']==tm])
    w.catchers = [k for k,v in w.h_dict.items() if 'C,' in v['all_pos']]
    print(tm)
    w._make_pitcher_combos()
    w._make_hitter_combos()
    print(w.pitcher_optimized_z, w.pitcher_optimized_lineup)
    print(w.hitter_optimized_z, w.hitter_optimized_lineup)
    return w

In [9]:
position_priority = ['C', '2B', '3B', 'SS', 'OF', '1B', 'MI', 'CI', 'DH', 'SP', 'RP']

def stitch_positions(row):
    pos_code = row[position_priority+['P']]>=5
    return list(pos_code[pos_code].index)

In [12]:
yr = datetime.now().year
wk = pd.read_sql(f"SELECT max(week) week FROM projections WHERE year={yr}", conn).iloc[0]['week']

print(yr, wk)
df  = pd.read_sql(f"SELECT distinct p.CBSNAME Player, o.owner Owner, r.pos Decision, j.*, e.*, \
                    CASE WHEN e.DH>=5 THEN 'h' ELSE 'p' END As type \
                    FROM roster r \
                    INNER JOIN projections j On (j.cbsid=r.cbsid) \
                    INNER JOIN players p On (r.cbsid=p.cbsid) \
                    INNER JOIN owners o On (r.owner_id=o.owner_id) \
                    INNER JOIN (SELECT cbsid, all_pos, posC C, pos1B '1B', pos2B '2B', pos3B '3B', posSS SS, posOF OF, \
                        posDH DH, posSP SP, posRP RP, posP P FROM eligibility WHERE year={yr} and week={wk-1}) e \
                        On (r.cbsid=e.cbsid) \
            WHERE j.year={yr} AND j.week={wk} AND j.proj_type='ros' AND r.year=2024 AND r.week={wk-1} \
            ORDER BY Owner, year, week", conn)

df2 = df.copy()
df[df['Owner']=='Lima Time!'].shape

2024 18


(33, 38)

In [281]:
yr = datetime.now().year
wk = pd.read_sql(f"SELECT max(week) week FROM projections WHERE year={yr}", conn).iloc[0]['week']-1
df  = pd.read_sql(f"SELECT distinct p.CBSNAME Player, o.owner Owner, r.pos Decision, j.*, e.*, \
                CASE WHEN e.DH>=5 THEN 'h' ELSE 'p' END As type \
                FROM roster r \
                INNER JOIN projections j On (j.cbsid=r.cbsid) \
                INNER JOIN players p On (r.cbsid=p.cbsid) \
                INNER JOIN owners o On (r.owner_id=o.owner_id) \
                INNER JOIN (SELECT cbsid, all_pos, posC C, pos1B '1B', pos2B '2B', pos3B '3B', posSS SS, posOF OF, posDH DH, posSP SP, posRP RP, posP P FROM eligibility WHERE year=2024 and week={wk-1}) e On (r.cbsid=e.cbsid) \
        WHERE j.year={yr} AND j.week={wk+1} AND j.proj_type='ros' AND r.year={yr} AND r.week={wk-1} \
        ORDER BY Owner, year, week", conn)
print(yr, wk)

2024 17


In [13]:
df[df['Owner']=='Lima Time!'][['Player', 'all_pos', 'z', 'type', 'HR', 'RBI', 'W']].sort_values('z', ascending=False)

,Player,all_pos,z,type,HR,RBI,W
176,Kyle Tucker,"OF,DH",6.062728,h,11.8202,35.3753,NaN
180,Garrett Crochet,"SP,P",5.566000,p,NaN,NaN,3.8138
160,Josh Naylor,"1B,CI,DH",4.896191,h,11.7658,39.0565,NaN
161,Ketel Marte,"2B,MI,DH",4.829651,h,9.1360,33.3575,NaN
171,Marcell Ozuna,DH,4.342127,h,14.2591,39.8290,NaN
170,Bryan Reynolds,"OF,DH",4.269944,h,10.4891,34.0285,NaN
167,Teoscar Hernandez,"OF,DH",2.890569,h,11.6135,36.3868,NaN
162,Alex Bregman,"3B,CI,DH",2.738694,h,9.5377,34.2818,NaN
164,Nico Hoerner,"2B,SS,MI,DH",2.730504,h,3.8741,22.7879,NaN
169,Brandon Nimmo,"OF,DH",2.517611,h,8.7588,29.8490,NaN


In [14]:
def optimize(tm, df):
    df = df.rename(columns={'owner':'Owner', 'CBSNAME':'Player'})
    w = ol.Optimized_Lineups(tm, df[df['Owner']==tm])
    w.catchers = [k for k,v in w.h_dict.items() if 'C,' in v['all_pos']]
    w._make_pitcher_combos()
    print(w.pitcher_optimized_z, w.pitcher_optimized_lineup)
    w._make_hitter_combos()
    print(w.hitter_optimized_z, w.hitter_optimized_lineup)
    return w

In [223]:
data = proj.copy()
owner = 'Lima Time!'
data[data['owner']==owner][['CBSNAME','all_pos', 'z', 'type']].sort_values(['type', 'z'], ascending=[True, False])

,CBSNAME,all_pos,z,type
17,Kyle Tucker,"OF,DH",6.070290,h
1,Josh Naylor,"1B,CI,DH",4.883640,h
2,Ketel Marte,"2B,MI,DH",4.584004,h
12,Marcell Ozuna,DH,4.221917,h
11,Bryan Reynolds,"OF,DH",3.942267,h
5,Nico Hoerner,"2B,SS,MI,DH",2.981359,h
8,Teoscar Hernandez,"OF,DH",2.852812,h
3,Alex Bregman,"3B,CI,DH",2.765098,h
10,Brandon Nimmo,"OF,DH",2.560245,h
6,Ryan Mountcastle,"1B,CI,DH",1.280574,h


In [64]:
df[df['Player'].isin(['Jurickson Profar', 'Jarren Duran', 'Nick Pivetta', 'Willi Castro', 'Joe Ryan', 'Ketel Marte'])]

,Player,Owner,Decision,cbsid,year,week,proj_type,AB,R,H,HR,RBI,BB,SB,BA,IP,W,SO,Sv+Hld,BBa,Ha,ERA,WHIP,ER,z,cbsid,all_pos,C,1B,2B,3B,SS,OF,DH,SP,RP,P,type
24,Joe Ryan,9 Grand Kids,SP,2933069,2024,18,ros,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,73.6649,4.7197,83.4929,0.0,16.9802,63.3394,3.61545,1.09034,29.5924,5.228942,2933069,"SP,P",0,0,0,0,0,0,0,18,0,5,p
161,Ketel Marte,Lima Time!,2B,1963334,2024,18,ros,245.068,39.4067,68.3792,9.1360,33.3575,26.9425,3.8961,0.279021,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,4.829651,1963334,"2B,MI,DH",0,0,5,0,1,0,5,0,0,0,h
197,Willi Castro,Mom's Cookin,MI,2171637,2024,18,ros,186.066,24.2798,45.9692,4.2021,19.7476,14.8369,8.2915,0.247058,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,-1.161785,2171637,"2B,3B,SS,MI,CI,OF,DH",0,0,5,5,5,5,5,0,0,0,h
202,Jurickson Profar,Mom's Cookin,OF,1796123,2024,18,ros,218.607,30.3357,54.8088,6.5526,26.3265,27.4160,2.4641,0.250718,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,-0.249406,1796123,"1B,CI,OF,DH",0,5,0,0,0,5,5,0,0,0,h
218,Nick Pivetta,Mom's Cookin,RP,2167338,2024,18,ros,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,72.8411,4.3077,80.0813,0.0,26.1876,66.9921,4.24665,1.27922,34.3701,0.126483,2167338,"SP,P",0,0,0,0,0,0,0,13,0,5,p
299,Jarren Duran,Ugly Spuds,OF,2932710,2024,18,ros,258.951,39.9827,67.7331,6.8726,28.2143,22.1489,13.8809,0.261567,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,4.950620,2932710,"OF,DH",0,0,0,0,0,5,5,0,0,0,h


In [225]:
proj.loc[(proj['all_pos']==0) & (proj['pos'].isin(['SP', 'RP', 'P'])), 'all_pos'] = 'P'
proj.loc[(proj['all_pos']==0) & (proj['pos'].isin(['C', '1B', '2B', '3B', 'SS', 'OF', 'MI', 'CI', 'DH'])), 'all_pos'] = proj.loc[proj['pos'].isin(['C', '1B', '2B', '3B', 'SS', 'OF', 'MI', 'CI', 'DH']), 'pos']+',DH'

In [17]:
#proj.loc[proj['CBSNAME']=='Dustin May', 'all_pos'] = 'P'
#proj.loc[proj['CBSNAME']=='Junior Caminero', 'all_pos'] = '3B,DH'
lt = optimize('Lima Time!', df)

18.911845509597722 ['Garrett Crochet', 'Cole Ragans', 'Shota Imanaga', 'Hunter Brown', 'Edwin Diaz', 'Yusei Kikuchi', 'Justin Steele', 'Andres Munoz', 'Bryan Abreu']
35.13213846194855 ['Yainer Diaz', 'Josh Naylor', 'Ketel Marte', 'Nico Hoerner', 'Alex Bregman', 'Xander Bogaerts', 'Ryan Mountcastle', 'Kyle Tucker', 'Bryan Reynolds', 'Teoscar Hernandez', 'Brandon Nimmo', 'TJ Friedl', 'Marcell Ozuna', 'Thairo Estrada']


In [23]:
df[df['Player'].isin(lt.hitter_optimized_lineup+lt.pitcher_optimized_lineup)].HR.sum()

125.1431

In [18]:
gk = optimize('9 Grand Kids', df)

21.980806003294205 ['George Kirby', 'Joe Ryan', 'Kevin Gausman', 'Carlos Rodon', 'Framber Valdez', 'Camilo Doval', 'Peter Fairbanks', 'Trevor Megill', 'Aaron Civale']
24.144041894635887 ['Salvador Perez', 'Jeimer Candelario', 'Ozzie Albies', 'Trea Turner', "Ke'Bryan Hayes", 'Ezequiel Tovar', 'Manny Machado', 'Steven Kwan', 'Ian Happ', 'Lourdes Gurriel', 'Brandon Marsh', 'Wilyer Abreu', 'Carlos Correa', 'Jesse Winker']


In [336]:
hw = optimize('Harveys Wallbangers', df)

13.521962078154784 ['Tarik Skubal', 'Grayson Rodriguez', 'Nick Lodolo', 'Jeffrey Springs', 'Hunter Greene', 'Kodai Senga', 'Hunter Harvey', 'Bryan Woo', 'Aroldis Chapman']
19.705506780569806 ["Logan O'Hoppe", 'Vinnie Pasquantino', 'Bryson Stott', 'C.J. Abrams', 'Gunnar Henderson', 'Corey Seager', 'Jake Burger', 'Julio Rodriguez', 'Corbin Carroll', 'Wyatt Langford', 'Michael Toglia', 'Evan Carter', 'Ty France', 'Nick Gonzales']


In [175]:
ch = optimize('Charmer', df)

17.978056668119947 ['Corbin Burnes', 'Logan Gilbert', 'Luis Castillo', 'Emmanuel Clase', 'Taj Bradley', 'Evan Phillips', 'Cristopher Sanchez', 'Jeff Hoffman', 'Reese Olson']
23.7102728885975 ['Will Smith', 'Christian Walker', 'Maikel Garcia', 'Mookie Betts', 'Royce Lewis', 'Luis Garcia', 'Mark Vientos', 'Jazz Chisholm', 'Randy Arozarena', 'Christopher Morel', 'Luis Rengifo', 'Fernando Tatis', 'Shohei Ohtani', 'Alex Verdugo']


In [32]:
df2 = df.copy()

NameError: name 'df' is not defined

In [31]:
df2.loc[df2['Player'].isin(['Nick Pivetta']), 'Owner'] = 'Lima Time!'
df2.loc[df2['Player'].isin(['Xander Bogaerts']), 'Owner'] = 'Charmer'

NameError: name 'df2' is not defined

In [334]:
df2.loc[df2['Player'].isin(['Jose Altuve']), 'Owner'] = 'Lima Time!'
df2.loc[df2['Player'].isin(['Teoscar Hernandez']), 'Owner'] = 'Harveys Wallbangers'

In [30]:
lt2 = optimize('Lima Time!', df2)

20.75876046171844 ['Garrett Crochet', 'Cole Ragans', 'Shota Imanaga', 'Hunter Brown', 'Edwin Diaz', 'Yusei Kikuchi', 'Emmanuel Clase', 'Justin Steele', 'Andres Munoz']
28.942711687471768 ['Yainer Diaz', 'Josh Naylor', 'Ketel Marte', 'Nico Hoerner', 'Alex Bregman', 'Xander Bogaerts', 'Ryan Mountcastle', 'Kyle Tucker', 'Teoscar Hernandez', 'Brandon Nimmo', 'TJ Friedl', 'Jacob Young', 'Marcell Ozuna', 'Thairo Estrada']


In [27]:
optimize('Charmer', df2)

ValueError: attempt to get argmax of an empty sequence

In [165]:
optimize('9 Grand Kids', df2)

8.299269088171982 ['Kevin Gausman', 'Carlos Rodon', 'George Kirby', 'Framber Valdez', 'Joe Ryan', 'Peter Fairbanks', 'Seth Lugo', 'Trevor Megill', 'Aaron Civale']
22.25828837544655 ['J.T. Realmuto', 'Salvador Perez', 'Ozzie Albies', 'Trea Turner', 'Manny Machado', 'Carlos Correa', "Ke'Bryan Hayes", 'Steven Kwan', 'Brandon Marsh', 'Michael Conforto', 'Wilyer Abreu', 'Lourdes Gurriel', 'Ezequiel Tovar', 'Jeimer Candelario']


In [58]:
df[df['Player'].isin(lt.hitter_optimized_lineup)][['HR', 'SB', 'R', 'RBI']].sum()

HR     125.1431
SB      58.5354
R      444.5586
RBI    432.4984
dtype: float64

In [59]:
df2[df2['Player'].isin(lt2.hitter_optimized_lineup)][['HR', 'SB', 'R', 'RBI']].sum()

HR     116.3940
SB      66.1346
R      427.3897
RBI    415.0580
dtype: float64

In [33]:
yg = optimize('Young Guns', df)

2.045286643508465 ['Josh Hader', 'Nathan Eovaldi', 'MacKenzie Gore', 'Clay Holmes', 'Chris Bassitt', 'Devin Williams', 'Jose Berrios', 'Charlie Morton', 'Dane Dunning']
14.231398327218153 ['William Contreras', 'Pete Alonso', 'Ryan McMahon', 'Brice Turang', 'Ha-seong Kim', 'Jeff McNeil', 'Matt Olson', 'Kyle Schwarber', 'Brenton Doyle', 'Michael Harris', 'Jackson Chourio', 'MJ Melendez', 'Rhys Hoskins', 'Connor Joe']


In [71]:
optimize('Dirty Birds', df)

15.57920801314403 ['Pablo Lopez', 'Ranger Suarez', 'Yusei Kikuchi', 'Jhoan Duran', 'Robert Suarez', 'Joe Musgrove', 'Bobby Miller', 'Raisel Iglesias', 'Jesus Luzardo']
6.764885894344239 ['Adley Rutschman', 'Brendan Donovan', 'Marcus Semien', 'Bo Bichette', 'Nolan Arenado', 'Justin Turner', 'Josh Bell', 'Adolis Garcia', 'Eloy Jimenez', 'Lane Thomas', 'Nick Castellanos', 'Joc Pederson', 'Jo Adell', 'Masyn Winn']


In [177]:
optimize('Brewbirds', df)

17.01276042797701 ['Chris Sale', 'Sonny Gray', 'Logan Webb', 'Ryan Helsley', 'David Bednar', 'Yoshinobu Yamamoto', 'Kenley Jansen', 'Jacob deGrom', 'Carlos Estevez']
33.30215082856864 ['Willson Contreras', 'Vladimir Guerrero', 'Andres Gimenez', 'Francisco Lindor', 'Austin Riley', 'Gleyber Torres', 'Jose Ramirez', 'Yordan Alvarez', 'Alec Burleson', 'James Wood', 'Jake Fraley', 'Will Benson', 'J.D. Martinez', 'Colt Keith']


In [60]:
tm = 'Ugly Spuds'
us = optimize(tm, df)

18.92976630649876 ['Paul Skenes', 'Tyler Glasnow', 'Blake Snell', 'David Robertson', 'Yimi Garcia', 'Bryce Miller', 'Shane Baz', 'Ryan Pressly', 'JoJo Romero']
15.737919306882137 ['Patrick Bailey', 'Spencer Steer', 'Zack Gelof', 'Oneil Cruz', 'Elly De La Cruz', 'Zachary Neto', 'Jordan Westburg', 'Jarren Duran', 'Christian Yelich', 'Riley Greene', 'Byron Buxton', 'Heliot Ramos', 'Cade Smith', 'Nolan Jones']


In [62]:
wt = optimize('Wiscompton Wu-Tang', df)

12.518821921319953 ['Ryan Pepiot', 'Bailey Ober', 'Daniel Hudson', 'Kutter Crawford', 'Jack Flaherty', 'Grayson Rodriguez', 'Kyle Bradish', 'Alek Manoah', 'Tanner Bibee']
16.958786739449124 ['Mitch Garver', 'Alec Bohm', 'Bryson Stott', 'Willy Adames', 'Gunnar Henderson', 'Brandon Lowe', 'Michael Busch', 'Ronald Acuna', 'Luis Robert', 'Brent Rooker', 'Mike Trout', 'Daulton Varsho', 'Javier Baez', 'Jonathan India']


In [32]:
ch = optimize('Charmer', df)

17.447822792734573 ['Luis Castillo', 'Emmanuel Clase', 'Corbin Burnes', 'Evan Phillips', 'Logan Gilbert', 'Reese Olson', 'Cristopher Sanchez', 'Jeff Hoffman', 'Taj Bradley']
25.414204973984333 ['Will Smith', 'Christian Walker', 'Luis Garcia', 'Mookie Betts', 'Royce Lewis', 'Luis Rengifo', 'Christopher Morel', 'Fernando Tatis', 'Jazz Chisholm', 'Kerry Carpenter', 'Randy Arozarena', 'Alex Verdugo', 'Shohei Ohtani', 'Noelvi Marte']


In [65]:
db = optimize('Dirty Birds', df)

12.139819691110032 ['Pablo Lopez', 'Ranger Suarez', 'Jhoan Duran', 'Robert Suarez', 'Joe Musgrove', 'Bobby Miller', 'Raisel Iglesias', 'Jesus Luzardo', 'Andrew Kittredge']
6.764885894344239 ['Adley Rutschman', 'Brendan Donovan', 'Marcus Semien', 'Bo Bichette', 'Nolan Arenado', 'Justin Turner', 'Josh Bell', 'Adolis Garcia', 'Eloy Jimenez', 'Lane Thomas', 'Nick Castellanos', 'Joc Pederson', 'Jo Adell', 'Masyn Winn']


In [66]:
tc = optimize('Trouble with the Curve', df)

8.484379046763081 ['Zac Gallen', 'Zach Eflin', 'Joel Payamps', 'Freddy Peralta', 'Max Scherzer', 'Dean Kremer', 'Craig Kimbrel', 'Luis Severino', 'Marcus Stroman']
21.465942811273592 ['Sean Murphy', "Ryan O'Hearn", 'Isaac Paredes', 'Bobby Witt', 'Yandy Diaz', 'Ceddanne Rafaela', 'Mark Canha', 'Juan Soto', 'Seiya Suzuki', 'Riley Greene', 'Cedric Mullins', 'George Springer', 'Max Kepler', 'Joseph Ortiz']


In [67]:
mc = optimize("Mom's Cookin", df)

ValueError: attempt to get argmax of an empty sequence

In [35]:
rr = optimize('Roid Ragers', df)

14.627731970225861 ['Zack Wheeler', 'Mason Miller', 'Gerrit Cole', 'Michael King', 'Tanner Houck', 'Luis Gil', 'Griffin Jax', 'Paul Sewald', 'Kutter Crawford']
25.17952338965405 ['Cal Raleigh', 'Bryce Harper', 'Luis Arraez', 'Jeremy Pena', 'Rafael Devers', 'Dansby Swanson', 'Jake Cronenworth', 'Aaron Judge', 'Taylor Ward', "Tyler O'Neill", 'Cody Bellinger', 'Bryan De La Cruz', 'Joshua Lowe', 'Johan Rojas']


In [34]:
df[(df['Start']==1)].groupby('Owner').apply(lambda s: pd.Series({
    'BA':round(s['H'].sum()/s['AB'].sum(),3),
    'HR':s['HR'].sum(),
    'SB':s['SB'].sum(),
    'R':s['R'].sum(),
    'RBI':s['RBI'].sum(),
    'W': s['W'].sum(),
    'SO':s['SO'].sum(),
    'Sv+Hld':s['S'].sum()+s['HD'].sum(),
    'ERA':round(s['ER'].sum()/s['IP'].sum()*9,2),
    'WHIP':round((s['Ha'].sum()+s['BBa'].sum())/s['IP'].sum(),2),
}))

KeyError: 'Start'